In [ ]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load data func (Updated for 3 Prompts)
def load_and_normalize(file_path, domain_type):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except FileNotFoundError:
        print(f"ERROR: Không tìm thấy file tại {file_path}")
        return pd.DataFrame()
    except json.JSONDecodeError:
        print(f"ERROR: File {file_path} lỗi định dạng JSON")
        return pd.DataFrame()

    normalized_data = []

    for entry in data:
        clean_entry = {}
        # Lowercase top-level keys để dễ truy xuất
        temp_entry = {k.lower(): v for k, v in entry.items()}

        food_attrs = temp_entry.get('food_attributes', {})
        img_attrs = temp_entry.get('image_attributes', {})

        # --- Helper Function: Extract Value Robustly ---
        def get_val(source_dict, key_name):
            # Tìm key không phân biệt hoa thường
            val = None
            for k, v in source_dict.items():
                if k.lower() == key_name.lower():
                    val = v
                    break

            if val is None: return "Unknown"

            # Xử lý nếu giá trị là String (Format mới từ prompt của bạn)
            if isinstance(val, str):
                return val

            # Xử lý nếu giá trị là List
            if isinstance(val, list):
                return val[0] if val else "Unknown"

            # Xử lý format cũ (Dictionary {"Label": true}) - Giữ lại để tương thích ngược
            if isinstance(val, dict):
                true_keys = [k for k, v in val.items() if v is True or v == "Present"]
                if true_keys: return true_keys[0]
                return "Unknown"

            return str(val)

        # --- A. COMMON FIELDS (Image Attributes) ---
        clean_entry['Dataset'] = domain_type
        # Tên file ảnh (thường nằm ở top level hoặc id)
        clean_entry['id_anh'] = temp_entry.get('id_anh', temp_entry.get('filename', 'unknown'))

        clean_entry['Lighting'] = get_val(img_attrs, 'lighting_condition')
        clean_entry['Background'] = get_val(img_attrs, 'background_complexity')
        clean_entry['Occlusion'] = get_val(img_attrs, 'occlusion_level')

        # --- B. DOMAIN SPECIFIC FIELDS (Food Attributes) ---

        # 1. Xử lý SUON (Cấu trúc Morphology lồng nhau)
        if domain_type == 'Suon':
            # Morphology của Suon là dict: {"category": "...", "detail_variation": "..."}
            # Ta cần lấy "category" ra làm Morphology chính
            morph_data = None
            for k, v in food_attrs.items():
                if k.lower() == 'morphology':
                    morph_data = v
                    break

            if isinstance(morph_data, dict):
                clean_entry['Morphology'] = morph_data.get('category', 'Unknown')
            elif isinstance(morph_data, str):
                clean_entry['Morphology'] = morph_data
            else:
                clean_entry['Morphology'] = 'Unknown'

            clean_entry['Texture'] = get_val(food_attrs, 'surface_texture')
            clean_entry['Specific_Context'] = get_val(food_attrs, 'dish_context')

        # 2. Xử lý CHA CA (Interaction_State)
        elif domain_type == 'Cha Ca':
            clean_entry['Morphology'] = get_val(food_attrs, 'Morphology')
            clean_entry['Texture'] = get_val(food_attrs, 'Surface_Texture')
            clean_entry['Specific_Context'] = get_val(food_attrs, 'Interaction_State')

        # 3. Xử lý TOFU (Dish_Variant)
        elif domain_type == 'Tofu':
            clean_entry['Morphology'] = get_val(food_attrs, 'Morphology')
            clean_entry['Texture'] = get_val(food_attrs, 'Surface_Texture')
            clean_entry['Specific_Context'] = get_val(food_attrs, 'Dish_Variant')

        normalized_data.append(clean_entry)

    return pd.DataFrame(normalized_data)

# 2. Process Data
# --- CẬP NHẬT ĐƯỜNG DẪN FILE CỦA BẠN TẠI ĐÂY ---
path_chaca = '/content/drive/MyDrive/Data MetaData/train_chaca_qwen_v3_final.json'
path_tofu = '/content/drive/MyDrive/Data MetaData/train_tofu_qwen_v3_cleaned_final.json'
path_suon = '/content/drive/MyDrive/Data MetaData/train_suon_full_v6_cleaned_final.json'

print("Đang load dữ liệu...")
df_chaca = load_and_normalize(path_chaca, 'Cha Ca')
df_tofu = load_and_normalize(path_tofu, 'Tofu')
df_suon = load_and_normalize(path_suon, 'Suon')

# Combine datasets
df_combined = pd.concat([df_chaca, df_tofu, df_suon], ignore_index=True)

if df_combined.empty:
    print("Cảnh báo: Không có dữ liệu nào được load. Kiểm tra lại đường dẫn file.")
else:
    print(f"Tổng số ảnh đã load: {len(df_combined)}")

    # 3. VISUALIZATION
    # Setup lưới đồ thị: 3 hàng
    # Hàng 1: Image Attributes (Chung)
    # Hàng 2: Food Attributes (Riêng từng món)
    # Hàng 3: Tổng quan

    fig = plt.figure(figsize=(20, 18))
    plt.subplots_adjust(hspace=0.6, wspace=0.3)

    # --- ROW 1: Common Image Attributes (Lighting, BG, Occlusion) ---
    # Sử dụng palette thống nhất để dễ nhìn
    common_palette = 'viridis'

    ax1 = plt.subplot(3, 3, 1)
    if 'Lighting' in df_combined.columns:
        sns.countplot(data=df_combined, x='Lighting', hue='Dataset', ax=ax1, palette='Set2')
        ax1.set_title('Lighting Condition')
        ax1.tick_params(axis='x', rotation=30)

    ax2 = plt.subplot(3, 3, 2)
    if 'Background' in df_combined.columns:
        sns.countplot(data=df_combined, x='Background', hue='Dataset', ax=ax2, palette='Set2')
        ax2.set_title('Background Complexity')
        ax2.tick_params(axis='x', rotation=30)

    ax3 = plt.subplot(3, 3, 3)
    if 'Occlusion' in df_combined.columns:
        sns.countplot(data=df_combined, x='Occlusion', hue='Dataset', ax=ax3, palette='Set2')
        ax3.set_title('Occlusion Level')
        ax3.tick_params(axis='x', rotation=30)

    # --- ROW 2: Domain Specific Attributes ---

    # Helper để plot attribute riêng
    def plot_specific(df, dataset_name, ax_obj, color_palette):
        if not df.empty:
            # Melt dữ liệu để gom Morphology, Texture, Context vào cùng 1 trục Y
            cols_to_plot = ['Morphology', 'Texture', 'Specific_Context']
            # Lọc bỏ các cột không tồn tại hoặc toàn Unknown
            valid_cols = [c for c in cols_to_plot if c in df.columns]

            df_melt = df[valid_cols].melt(var_name='Attribute Type', value_name='Value')
            # Loại bỏ giá trị Unknown để biểu đồ sạch hơn (tùy chọn)
            df_melt = df_melt[df_melt['Value'] != 'Unknown']

            sns.countplot(data=df_melt, y='Value', hue='Attribute Type', ax=ax_obj, dodge=False, palette=color_palette)
            ax_obj.set_title(f'{dataset_name} Specific Attributes')
            ax_obj.set_xlabel('Count')
            ax_obj.legend(loc='lower right', title='Attr Type')

    # Plot Cha Ca
    ax4 = plt.subplot(3, 3, 4)
    plot_specific(df_chaca, 'Cha Ca', ax4, 'Reds')

    # Plot Tofu
    ax5 = plt.subplot(3, 3, 5)
    plot_specific(df_tofu, 'Tofu', ax5, 'Blues')

    # Plot Suon
    ax6 = plt.subplot(3, 3, 6)
    plot_specific(df_suon, 'Suon', ax6, 'Oranges')

    # --- ROW 3: Overall Stats ---

    # Data Distribution Pie Chart
    ax7 = plt.subplot(3, 3, 😎
    dataset_counts = df_combined['Dataset'].value_counts()
    if not dataset_counts.empty:
        ax7.pie(dataset_counts, labels=dataset_counts.index, autopct='%1.1f%%', colors=sns.color_palette('Set2'))
        ax7.set_title('Dataset Distribution')

    plt.tight_layout()
    plt.savefig('multimodal_food_analysis.png') # Lưu file ảnh
    plt.show()

    # 4. In ra thống kê text
    print("\n=== TOP FEATURES SUMMARY ===")
    for name, df in [('Cha Ca', df_chaca), ('Tofu', df_tofu), ('Suon', df_suon)]:
        if not df.empty:
            print(f"\n--- {name} ---")
            print(f"Top Context: {df['Specific_Context'].mode()[0] if not df['Specific_Context'].empty else 'N/A'}")
            print(f"Top Morphology: {df['Morphology'].mode()[0] if not df['Morphology'].empty else 'N/A'}")